In [29]:
%pip install python-dotenv langchain langchain-text-splitters langchain_pinecone
%pip install -qU pymupdf
%pip install --upgrade pip


  Using cached langchain_pinecone-0.2.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.5 kB)
  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
  Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
Using cached langchain_pinecone-0.2.0-py3-none-any.whl (11 kB)
Using cached aiohttp-3.9.5-cp312-cp312-macosx_11_0_arm64.whl (392 kB)
Using cached pinecone_client-5.0.1-py3-none-any.whl (244 kB)
Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl (85 kB)
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to us

In [30]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
loader = PyMuPDFLoader('./year-end-tax-settlement-guide-2024.pdf')
docs = loader.load()

document_list = text_splitter.split_documents(docs)

/Users/a13517/.pyenv/versions/3.12.7/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 15 of document ./year-end-tax-settlement-guide-2024.pdf
  warnings.warn(
/Users/a13517/.pyenv/versions/3.12.7/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 37 of document ./year-end-tax-settlement-guide-2024.pdf
  warnings.warn(
/Users/a13517/.pyenv/versions/3.12.7/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 223 of document ./year-end-tax-settlement-guide-2024.pdf
  warnings.warn(


In [32]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone
from dotenv import load_dotenv

load_dotenv()

embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
index_name = 'year-end-tax-settlement-index'
database = Pinecone.from_documents(
    documents=document_list,
    embedding=embeddings,
    index_name=index_name
)


In [33]:
query = '연말정산을 준비하기 위해 필요한 서류는 어떤 것이 있나요?'

retriever = database.as_retriever()
retrieved_docs = retriever.invoke(query)

In [34]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

prompt = f"""[Identity]
- 당신은 회사에서 연말정산을 담당하고 있는 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""

ai_message = llm.invoke(prompt)

In [35]:
ai_message.content

'연말정산을 준비하기 위해 필요한 서류는 다음과 같습니다:\n\n1. **소득·세액공제 증명서류**: \n   - 연말정산간소화 서비스를 통해 제공되는 소득·세액공제 자료를 출력하거나 전산 파일로 내려받아 준비합니다.\n   - 연말정산간소화 서비스에서 제공되지 않는 서류는 근로자가 직접 수집해야 합니다. 예를 들어, 난임시술비, 취학 전 아동 학원비 등이 이에 해당합니다.\n\n2. **기부금명세서**: 기부금 공제를 받기 위해 필요한 서류입니다.\n\n3. **의료비지급명세서**: 의료비 공제를 받기 위해 필요한 서류입니다.\n\n4. **신용카드 등 소득공제 신청서**: 신용카드 사용에 대한 소득공제를 받기 위해 필요한 서류입니다.\n\n5. **월세액 세액공제 명세서**: 월세 공제를 받기 위해 임대차 계약서의 주소지 및 계약기간 등의 상세 내역을 포함한 서류가 필요합니다.\n\n이 외에도 특정 상황에 따라 추가적인 서류가 필요할 수 있습니다. 예를 들어, 외국인 기술자 세액감면, 중소기업 취업자 소득세 감면 신청서 등이 있습니다. 각 공제 항목에 따라 필요 서류를 정확히 준비하여 제출하는 것이 중요합니다.'